<a href="https://colab.research.google.com/github/DarianSawali/News-Based-RAG/blob/main/News_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feedparser beautifulsoup4 sentence-transformers faiss-cpu transformers python-dateutil scikit-learn requests


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 78.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=73505f15fe493b7dea9bb3dd88a26549976da2e99f8201ac849aaa1bf7005e8c
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [2]:
import re
import requests
import feedparser
from bs4 import BeautifulSoup
from datetime import timezone
from dateutil import parser as dateparser

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer
import faiss

from transformers import GPT2LMHeadModel, GPT2Tokenizer


In [3]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
gpt2 = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")

tokenizer.pad_token = tokenizer.eos_token

print("Models loaded.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Models loaded.


In [4]:
def clean_text(raw_html: str) -> str:
    """
    Clean RSS summary / HTML content:
    - remove tags
    - drop images (and their alt text/captions)
    - collapse whitespace
    """
    if not raw_html:
        return ""

    soup = BeautifulSoup(raw_html, "html.parser")

    for img in soup.find_all("img"):
        img.decompose()

    text = soup.get_text(separator=" ")

    text = re.sub(r"\s+", " ", text).strip()
    return text


In [5]:
def safe_feedparse(url, timeout=5):
    """Fetch RSS safely with timeout and error handling."""
    try:
        resp = requests.get(url, timeout=timeout, headers={"User-Agent": "Mozilla/5.0"})
        resp.raise_for_status()
        return feedparser.parse(resp.text)
    except Exception as e:
        print("Failed to fetch:", url, "→", e)
        return feedparser.parse("")

def fetch_articles_from_rss(source_name, feed_url, max_articles=20):
    feed = safe_feedparse(feed_url)
    print(source_name, "| entries in feed:", len(feed.entries))

    docs = []
    for entry in feed.entries[:max_articles]:
        title = entry.get("title", "").strip()
        summary = entry.get("summary", "").strip()
        url = entry.get("link", "")

        clean_summary = clean_text(summary)
        text = clean_summary or title
        if not text:
            continue

        docs.append({
            "source": source_name,
            "title": title,
            "text": text,
            "url": url,
            "published": entry.get("published", "")
        })

    return docs


In [6]:
NEWS_SOURCES = {
    "Global News BC": "https://globalnews.ca/bc/feed/",
    "CBC BC": "https://www.cbc.ca/webfeed/rss/rss-canada-britishcolumbia",
    "The Province": "https://theprovince.com/feed",
}


In [7]:
all_documents = []

for name, url in NEWS_SOURCES.items():
    docs = fetch_articles_from_rss(name, url, max_articles=20)
    print(" → Retrieved:", len(docs))
    all_documents.extend(docs)

print("Total articles:", len(all_documents))


Global News BC | entries in feed: 10
 → Retrieved: 10
CBC BC | entries in feed: 20
 → Retrieved: 20
The Province | entries in feed: 10
 → Retrieved: 10
Total articles: 40


In [8]:
len(all_documents), all_documents[0]


(40,
 {'source': 'Global News BC',
  'title': 'Advocates push for judicial change after murder of Kelowna woman',
  'text': 'James Plover has been charged with first-degree murder in the death of his estranged wife, Bailey McCourt. The charge is now before the courts and no verdicts have been made.',
  'url': 'https://globalnews.ca/news/11548120/push-for-judicial-change-murder-kelowna-woman/',
  'published': 'Fri, 28 Nov 2025 01:22:07 +0000'})

In [9]:
corpus_texts = [
    doc["title"] + "\n\n" + doc["text"]
    for doc in all_documents
]

corpus_embeddings = embed_model.encode(corpus_texts, convert_to_numpy=True).astype("float32")

embedding_dim = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(corpus_embeddings)

print("FAISS index ready. Documents indexed:", len(corpus_texts))


FAISS index ready. Documents indexed: 40


In [10]:
def retrieve_docs(query, k=3):
    """Retrieve top-k similar documents for a query using FAISS."""
    q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(q_emb, k)

    docs = []
    for dist, idx in zip(distances[0], indices[0]):
        doc = all_documents[idx].copy()
        doc["distance"] = float(dist)
        docs.append(doc)

    return docs


In [11]:
for d in retrieve_docs("pipeline northern BC", k=3):
    print(d["source"], "-", d["title"], "| distance:", d["distance"])


CBC BC - Proposed pipeline through B.C. can’t distract from province’s other projects, Eby says | distance: 0.8235334157943726
CBC BC - Two key North Coast B.C. leaders meet with Alberta premier ahead of federal pipeline MOU | distance: 0.8447215557098389
Global News BC - B.C. premier calls pipeline MOU an ‘energy vampire’, First Nations call it a ‘pipe dream’ | distance: 0.9557517170906067


In [12]:
def extractive_summary(text, n_sentences=3):
    """
    Simple extractive summarizer:
    - split text into sentences
    - embed each
    - rank by similarity to whole doc
    """
    text = clean_text(text)

    sentences = re.split(r'(?<=[.!?])\s+', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    if not sentences:
        return ""

    if len(sentences) <= n_sentences:
        return " ".join(sentences)

    sent_embs = embed_model.encode(sentences, convert_to_numpy=True)
    doc_emb = embed_model.encode([text], convert_to_numpy=True)[0]

    sims = cosine_similarity(sent_embs, doc_emb.reshape(1, -1)).flatten()
    top_idx = np.argsort(sims)[-n_sentences:]
    top_idx = sorted(top_idx)

    selected = [sentences[i] for i in top_idx]
    return " ".join(selected)


In [13]:
def summarize_doc(doc, n_sentences=2):
    source = doc["source"]
    title = doc["title"]
    text = doc["text"]
    url = doc.get("url", "")

    short = extractive_summary(text, n_sentences=n_sentences)
    if not short:
        return "No content to summarize."

    line = f"{short} (Source: {source} — {title})"
    if url:
        line += f"\nLink: {url}"
    return line


In [14]:
sample = all_documents[0]
print(summarize_doc(sample))


James Plover has been charged with first-degree murder in the death of his estranged wife, Bailey McCourt. The charge is now before the courts and no verdicts have been made. (Source: Global News BC — Advocates push for judicial change after murder of Kelowna woman)
Link: https://globalnews.ca/news/11548120/push-for-judicial-change-murder-kelowna-woman/


In [15]:
def get_published_datetime(doc):
    pub = doc.get("published", "")
    if not pub:
        return None
    try:
        dt = dateparser.parse(pub)
        if dt is None:
            return None
        if dt.tzinfo is not None:
            dt = dt.astimezone(timezone.utc).replace(tzinfo=None)
        return dt
    except Exception:
        return None

def summarize_latest_news(n=5):
    dated_docs = []
    for d in all_documents:
        dt = get_published_datetime(d)
        if dt is not None:
            dated_docs.append((dt, d))

    if not dated_docs:
        return "No recent articles available."

    dated_docs.sort(key=lambda x: x[0], reverse=True)
    top_docs = [d for _, d in dated_docs[:n]]

    summaries = []
    for doc in top_docs:
        summaries.append(summarize_doc(doc, n_sentences=2))

    if not summaries:
        return "No recent articles available."

    return "\n\n".join(summaries)


In [16]:
print(summarize_latest_news(n=5))


James Plover has been charged with first-degree murder in the death of his estranged wife, Bailey McCourt. The charge is now before the courts and no verdicts have been made. (Source: Global News BC — Advocates push for judicial change after murder of Kelowna woman)
Link: https://globalnews.ca/news/11548120/push-for-judicial-change-murder-kelowna-woman/

Bajwa hopes to set the record by the time she’s 25, so it’s not a rush. There’s still time to savor the views from the summits and the little moments on every single trip. (Source: Global News BC — B.C. woman wants to become youngest Canadian to climb the 7 summits)
Link: https://globalnews.ca/news/11548196/bc-woman-become-youngest-canadian-climb-7-summits/

Canada’s South Asian communities, including those in Surrey, B.C., have seen a wave of extortion-related threats and violence, with British Columbia establishing a special team. (Source: Global News BC — B.C. premier, federal minister to announce strengthened response to extortion 

/usr/local/lib/python3.12/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [17]:
def search_articles(query, k=5):
    retrieved = retrieve_docs(query, k=k)
    if len(retrieved) == 0:
        print("No relevant news found for this query.")
        return []

    for idx, doc in enumerate(retrieved):
        print(f"[{idx}] {doc['title']} ({doc['source']})")
        if doc.get("published"):
            print("     Published:", doc["published"])
        if doc.get("url"):
            print("     Link:", doc["url"])
        print()
    return retrieved


In [18]:
candidates = search_articles("pipeline northern BC", k=5)


[0] Proposed pipeline through B.C. can’t distract from province’s other projects, Eby says (CBC BC)
     Published: Thu, 27 Nov 2025 11:12:10 EST
     Link: https://www.cbc.ca/news/politics/livestory/alberta-ottawa-memorandum-of-understanding-energy-deal-pipeline-bc-9.6993431?cmp=rss

[1] Two key North Coast B.C. leaders meet with Alberta premier ahead of federal pipeline MOU (CBC BC)
     Published: Wed, 26 Nov 2025 18:03:00 EST
     Link: https://www.cbc.ca/news/canada/british-columbia/kitimat-haisla-alberta-oil-pipeline-9.6993855?cmp=rss

[2] B.C. premier calls pipeline MOU an ‘energy vampire’, First Nations call it a ‘pipe dream’ (Global News BC)
     Published: Thu, 27 Nov 2025 19:35:00 +0000
     Link: https://globalnews.ca/news/11546713/bc-reaction-david-eby-pipeline-first-nations/

[3] Smaller B.C. Ferries vessel on north Vancouver Island route prompts local residents' anger (CBC BC)
     Published: Thu, 27 Nov 2025 08:00:00 EST
     Link: https://www.cbc.ca/news/canada/british

In [46]:
def tidy_summary(text: str) -> str:
    """
    Light cleanup for readability.
    - Fix common run-ons like 'says The' → 'says. The'
    - Collapse weird extra spaces
    """
    text = re.sub(r"\b(says|said)\s+([A-Z])", r"\1. \2", text)

    text = re.sub(r"\s+", " ", text).strip()
    return text


In [47]:
def safe_retrieve(query, k=3):
    """Retrieve docs, with a simple keyword fallback if nothing is found."""
    docs = retrieve_docs(query, k=k)
    if docs:
        return docs

    keywords = query.lower().split()
    alt_docs = []
    for kw in keywords:
        alt_docs.extend(retrieve_docs(kw, k=1))

    seen = set()
    unique = []
    for d in alt_docs:
        key = d.get("url") or d.get("title")
        if key not in seen:
            seen.add(key)
            unique.append(d)

    return unique[:k]

def fused_summary(docs, n_sentences=4):
    """
    Combine multiple articles into ONE clean extractive summary.
    No hallucinations. No broken sentences. No image alt-text.
    """

    combined_text = []
    for doc in docs:
        combined_text.append(doc["text"])
    merged = " ".join(combined_text)

    extracted = extractive_summary(merged, n_sentences=n_sentences)

    extracted = tidy_summary(extracted)

    sources_block = "\n".join([
        f"- {d['source']} — {d['title']} ({d.get('url','')})"
        for d in docs
    ])

    final = (
        f"{extracted}\n\n"
        f"Sources:\n{sources_block}"
    )

    return final


In [48]:
def summarize_query_with_rag(query, k=3):
    retrieved = safe_retrieve(query, k=k)

    if not retrieved:
        return f"No articles match your query: “{query}”"

    return fused_summary(retrieved, n_sentences=4)


In [49]:
print(summarize_query_with_rag("coastal first nations and the tanker ban", k=3))


The West Coast Oil Tanker Ban came into effect in 2019 and prohibits tankers from carrying more than 12,500 metric tonnes of crude oil along the northern coast of B.C. ʼNamgis First Nation elected Coun. Ernest Alfred says the temporary vessel, the Quadra Queen II, is too small to accommodate Port McNeill, Alert Bay and Sointula — small, intertwined communities about a five-hour drive north of Victoria. The mayor of Kitimat, B.C., and elected chief of the Haisla First Nation say they reminded Alberta Premier Danielle Smith of the longstanding opposition to an oil pipeline in the region.

Sources:
- Global News BC — B.C. premier calls pipeline MOU an ‘energy vampire’, First Nations call it a ‘pipe dream’ (https://globalnews.ca/news/11546713/bc-reaction-david-eby-pipeline-first-nations/)
- CBC BC — Smaller B.C. Ferries vessel on north Vancouver Island route prompts local residents' anger (https://www.cbc.ca/news/canada/british-columbia/bc-ferries-north-vancouver-island-port-mcneill-9.6993

In [50]:
def rewrite_with_gpt2(summary, max_new_tokens=40):
    """
    Use GPT-2 to lightly rewrite the extractive summary for readability.
    We prepend strong instructions to avoid adding new facts, but GPT-2 can still hallucinate.
    """
    prompt = (
        "Rewrite the following news summary in clear, fluent English. "
        "Do NOT add any new information or details that are not already stated.\n\n"
        f"SUMMARY:\n{summary}\n\nREWRITE:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    output_ids = gpt2.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated = output_ids[0][inputs["input_ids"].shape[1]:]
    rewritten = tokenizer.decode(generated, skip_special_tokens=True).strip()
    return rewritten


In [51]:
raw = summarize_query_with_rag("coastal first nations and the tanker ban", k=3)
print("EXTRACTIVE SUMMARY:\n", raw)

print("\n\nGPT-2 REWRITE (optional):\n", rewrite_with_gpt2(raw))


EXTRACTIVE SUMMARY:
 The West Coast Oil Tanker Ban came into effect in 2019 and prohibits tankers from carrying more than 12,500 metric tonnes of crude oil along the northern coast of B.C. ʼNamgis First Nation elected Coun. Ernest Alfred says the temporary vessel, the Quadra Queen II, is too small to accommodate Port McNeill, Alert Bay and Sointula — small, intertwined communities about a five-hour drive north of Victoria. The mayor of Kitimat, B.C., and elected chief of the Haisla First Nation say they reminded Alberta Premier Danielle Smith of the longstanding opposition to an oil pipeline in the region.

Sources:
- Global News BC — B.C. premier calls pipeline MOU an ‘energy vampire’, First Nations call it a ‘pipe dream’ (https://globalnews.ca/news/11546713/bc-reaction-david-eby-pipeline-first-nations/)
- CBC BC — Smaller B.C. Ferries vessel on north Vancouver Island route prompts local residents' anger (https://www.cbc.ca/news/canada/british-columbia/bc-ferries-north-vancouver-islan

In [52]:
def answer_news_query(query):
    q = query.lower().strip()

    if any(word in q for word in ["latest", "recent", "today", "this morning", "this evening"]):
        return summarize_latest_news(n=5)

    if q.startswith("summarize article") or q.startswith("summarize story"):
        stripped = (
            q.replace("summarize article", "")
             .replace("summarize story", "")
             .strip()
        )
        if not stripped:
            stripped = query
        return summarize_query_with_rag(query, k=3)

def chat_loop():
    print("Vancouver News Assistant. Type 'quit' to exit.\n")
    while True:
        q = input("You: ")
        if q.lower().strip() in ["quit", "exit"]:
            print("Bye!")
            break
        answer = answer_news_query(q)
        print("\nAssistant:\n" + answer + "\n")


In [53]:
print(answer_news_query("What are the latest stories?"))

print("\n---\n")
print(answer_news_query("summarize story about the B.C. pipeline MOU"))

print("\n---\n")
print(answer_news_query("coastal First Nations and the tanker ban"))



James Plover has been charged with first-degree murder in the death of his estranged wife, Bailey McCourt. The charge is now before the courts and no verdicts have been made. (Source: Global News BC — Advocates push for judicial change after murder of Kelowna woman)
Link: https://globalnews.ca/news/11548120/push-for-judicial-change-murder-kelowna-woman/

Bajwa hopes to set the record by the time she’s 25, so it’s not a rush. There’s still time to savor the views from the summits and the little moments on every single trip. (Source: Global News BC — B.C. woman wants to become youngest Canadian to climb the 7 summits)
Link: https://globalnews.ca/news/11548196/bc-woman-become-youngest-canadian-climb-7-summits/

Canada’s South Asian communities, including those in Surrey, B.C., have seen a wave of extortion-related threats and violence, with British Columbia establishing a special team. (Source: Global News BC — B.C. premier, federal minister to announce strengthened response to extortion 

In [54]:
print(answer_news_query("summarize story about the B.C. pipeline MOU"))


The West Coast Oil Tanker Ban came into effect in 2019 and prohibits tankers from carrying more than 12,500 metric tonnes of crude oil along the northern coast of B.C. Proposed pipeline through B.C. can’t distract from province’s other projects, Eby says. The mayor of Kitimat, B.C., and elected chief of the Haisla First Nation say they reminded Alberta Premier Danielle Smith of the longstanding opposition to an oil pipeline in the region.

Sources:
- Global News BC — B.C. premier calls pipeline MOU an ‘energy vampire’, First Nations call it a ‘pipe dream’ (https://globalnews.ca/news/11546713/bc-reaction-david-eby-pipeline-first-nations/)
- CBC BC — Proposed pipeline through B.C. can’t distract from province’s other projects, Eby says (https://www.cbc.ca/news/politics/livestory/alberta-ottawa-memorandum-of-understanding-energy-deal-pipeline-bc-9.6993431?cmp=rss)
- CBC BC — Two key North Coast B.C. leaders meet with Alberta premier ahead of federal pipeline MOU (https://www.cbc.ca/news/c

In [ ]:
# chat_loop()